# Model Analysis

In [ ]:
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Number of GPUs Available: ", len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
import os
import random
import glob
import numpy as np
import pandas as pd
import cv2
from PIL import Image, ImageChops, ImageEnhance, ImageFilter, ImageOps
from io import BytesIO
from scipy import ndimage, fftpack
import optuna
import tensorflow as tf
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report, roc_auc_score, precision_recall_curve, roc_curve, auc
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization, Reshape, Concatenate, Conv2DTranspose, UpSampling2D, Input
import matplotlib.pyplot as plt

from preprocess import *
from model import *
from utils import *

In [ ]:
def build_model(image_size=(128, 128)):
    filters =  32
    dropout_rate = 0.25
    model = Sequential()
    model.add(Conv2D(filters=filters, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(image_size[0], image_size[1], 3)))
    model.add(Conv2D(filters=filters, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(dropout_rate))

    model.add(Conv2D(filters=2*filters, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(filters=2*filters, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(dropout_rate))

    model.add(Conv2D(filters=4*filters, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(filters=4*filters, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(dropout_rate))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))

    model.summary()

    return model

In [ ]:
def train_model(model, train_generator, val_generator, batch_size, epochs):
    init_lr = 1e-4
   
    opt = Adam(learning_rate= init_lr, decay=init_lr / epochs)

    model.compile(optimizer= opt, loss='categorical_crossentropy', metrics=['accuracy'])

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, patience=5, verbose=0, mode='auto')
    checkpoint = tf.keras.callbacks.ModelCheckpoint('../models/model.h5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
    
    tensorboard_callback = TensorBoard(log_dir='logs', histogram_freq=1)
    
    model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=epochs,
        validation_data=val_generator,
        validation_steps=len(val_generator),
        callbacks=[early_stopping, checkpoint, tensorboard_callback]
    )
    #Obtener metricas de entrenamiento y validacion
    train_metrics = model.evaluate(train_generator, steps=len(train_generator), verbose=0)
    val_metrics = model.evaluate(val_generator, steps=len(val_generator), verbose=0)
    print('Train: %.3f, %.3f' % (train_metrics[0], train_metrics[1]))
    print('Validation: %.3f, %.3f' % (val_metrics[0], val_metrics[1]))

    return model

In [ ]:
if __name__ == '__main__':
    real_images_path = '../data/dataset/data/CASIA2/Au/*.*'
    fake_images_path = '../data/dataset/data/CASIA2/Tp/*.*'
    

    batch_size = 32
    epochs = 50
    image_size = (128, 128)
    X = []
    y = []

    print("Cargando datos de imágenes reales...")
    for file_path in glob.glob(real_images_path):
        X.append(preparete_image_ela(file_path, image_size))
        y.append(0)

    random.shuffle(X)

    print("Cargando datos de imágenes falsificadas...")
    for file_path in glob.glob(fake_images_path):
        X.append(preparete_image_ela(file_path, image_size))
        y.append(1)

    X= np.array(X).reshape(-1, image_size[0], image_size[1], 3)
    y = to_categorical(y, num_classes = 2)

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=5)
    X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=5)

    train_generator = ImageDataGenerator().flow(X_train, y_train, batch_size=batch_size)
    val_generator = ImageDataGenerator().flow(X_val, y_val, batch_size=batch_size)

    model = build_model(image_size)
    model = train_model(model, train_generator, val_generator, batch_size, epochs)

    #Calculamos la presicion del modelo con los datos de entrenamiento
    score = model.evaluate(X_train, y_train)
    print('TRAIN loss:', score[0])
    print('TRAIN accuracy:', score[1])

    # Calculamos la precisión del modelo con los datos de test
    score = model.evaluate(X_test, y_test)
    print('TEST loss:', score[0])
    print('TEST accuracy:', score[1])

    # Guardamos el modelo
    model.save('../models/model'+str(score[1])+'.h5')

In [ ]:
def build_model(image_size):
    dropout = 0.25
    model = Sequential()
    model.add(Conv2D(filters=30, kernel_size=(5, 5), padding='same', activation='relu', input_shape=(image_size[0], image_size[1], 3)))
    model.add(Conv2D(filters=30, kernel_size=(5, 5), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(dropout))

    model.add(Conv2D(filters=60, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(filters=60, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(dropout))

    model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(dropout))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))

    return model